## Import

In [ ]:
import os
import yaml
import warnings
from typing import List

from utils import get_openai_api_key
from utils_data import add_slot, is_slot_conflict, remove_slot, access_timetable, access_timetable_with_date

import crewai
from crewai import Agent, Task, Crew
from pydantic import BaseModel, Field
from crewai.tools import BaseTool


warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'backend'

In [ ]:
print(os.environ["OPENAI_MODEL_NAME"])

gpt-4o-mini


In [ ]:
#TIMETABLE = {}

"""{'06/10/2025': [{'weekday': 'Monday',
   'startTime': '17:00',
   'endTime': '18:00',
   'title': 'Meeting with Boss'}],
 '13/10/2025': [{'weekday': 'Monday',
   'startTime': '17:00',
   'endTime': '18:00',
   'title': 'Meeting with Boss'}]}"""

"""
    "date": [{"startTime", "endTime", "title"}, {...}, {...}, ...],
    "anotherdate": [...], 
"""

'\n    "date": [{"startTime", "endTime", "title"}, {...}, {...}, ...],\n    "anotherdate": [...], \n'

#### Get api key

In [ ]:
openai_api_key = get_openai_api_key()

## Setup configs

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Pydantic models

In [ ]:
class EventSlot(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")
    startTime: str = Field(..., description="Start time of the slot. The format is: HH:MM")
    endTime: str = Field(..., description="End time of the slot. The format is: HH:MM")
    title: str = Field(..., description="Short title to summarize the event's essential information.")

class MultipleSlots(BaseModel):
    slots: List[EventSlot] = Field(..., description="List of event slots")

## Tools

In [ ]:
from datetime import date
from typing import Type


from datetime import datetime

class InputDate(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")

class ValidDateTool(BaseTool):
    name: str = "Valid Date Tool"
    description: str = "Tool that checks if the date and week day are valid"
    args_schema: Type[BaseModel] = InputDate

    def _run(self, date: str, weekday: str):
        try:
            datetime.strptime(date, "%d/%m/%Y")
        except ValueError:
            return f"Error: {str(e)}. Please provide a valid date."
        
        try:
            date_obj = datetime.strptime(date, "%d/%m/%Y")
            correct_weekday = date_obj.strftime("%A")  # Full name (e.g., "Monday")
            correct_weekday.lower().startswith(weekday.lower())
            return "Valid date"
        except ValueError as e:
            return f"Error: {str(e)}. The week day does not corresponds to the date"
        
class AddSlot(BaseTool):
    name: str = "Add Slot Tool"
    description: str = "Tool that adds slots to the timetable using these informations: date, weekday, start time, end time and the title of the event."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        for eventSlot in slots:
            res = add_slot(eventSlot)
            if res != "SUCCESS":
                return res
        return "Successfully added slot(s)"
    
class RemoveSlot(BaseTool):
    name: str = "Remove Slot Tool"
    description: str = "Tool that removes slots to the timetable using these informations: date, weekday, start time, end time and the title of the event.\n" \
    "Do not call this tool again if already successful."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        res = ""
        for eventSlot in slots:
            res += remove_slot(eventSlot) + "\n"
        return res

class AccessTimetable(BaseTool):
    name: str = "Access Timetable"
    description: str = "Tool that enables access to slots on a specific date in dict (format of date DD/MM/YYYY)"
    
    def _run(self, date_input: str):
        return access_timetable_with_date(date_input)
            
    
class CheckConflict(BaseTool):
    name: str = "Check Conflict Tool"
    description: str = "Tool that checks if there is a conflict between an event and an already setup event in the timetable. " \
    "Returns the list of events where there is a conflict with already setup events in the timetable."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        conflicts = []
        for slot in slots:
            res = is_slot_conflict(slot)
            if res == "There is a conflict":
                conflicts.append(slot)
        return conflicts


## Agents

In [ ]:
converter = Agent(
    config=agents_config["converter"],
    inject_date=True,
    date_format="%A, %B %d, %Y",
    tools=[ValidDateTool()])

assistant = Agent(
    config=agents_config["assistant"],
    tools=[AddSlot(), CheckConflict(), RemoveSlot(), AccessTimetable()]
)

## Tasks

In [ ]:
conversion = Task(
    config=tasks_config["conversion"],
    agent=converter,
    output_pydantic=MultipleSlots
)

manage_timetable = Task(
    config=tasks_config["manage_timetable"],
    agent=assistant
)

## Crew

In [ ]:
crew = Crew(
    agents=[converter, assistant],
    tasks=[conversion, manage_timetable],
    verbose=False
)

## Execution

In [ ]:
inputs_array = [{"event": "I have a meeting with my boss for the next two mondays at 5pm for an hour (6th and 13th october)"},
                {"event": "Can you delete the meeting on the 6th october."},
                {"event": "Camille's birthday is on the 15/05/2025."}]

In [ ]:
result = crew.kickoff_for_each(inputs=inputs_array)

In [ ]:
result

[CrewOutput(raw='The slots for the meetings with your boss have been successfully added for the following dates: 6th October 2023 from 5:00 PM to 6:00 PM and 13th October 2023 from 5:00 PM to 6:00 PM.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Gather the most crucial information about the event: I have a meeting with my boss for the next two mondays at 5pm for an hour (6th and 13th october). You should gather the following if there if:\n    1. Date in format DD/MM/YYYY\n    2. Start time in format HH:MM\n    3. End time in format HH:MM\n    4. Event title, generate if not explicit\nMake sure you are sending a valid date.\n\n\nCurrent Date: Wednesday, October 01, 2025', name='Gather the most crucial information about the event: I have a meeting with my boss for the next two mondays at 5pm for an hour (6th and 13th october). You should gather the following if there if:\n    1. Date in format DD/MM/YYYY\n    2. Start time in format HH:MM\n    3. End time in for

## Cost

In [ ]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0033


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,22016,20691,4608,1325,19


In [ ]:
1

1